In [3]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
import pandas as pd

In [4]:
data_path = '/Users/zhangzongkun/Desktop/research/video ordering/cataract-101/videos/'

In [5]:
annotation_df = pd.read_csv('/Users/zhangzongkun/Desktop/research/video ordering/cataract-101/annotations.csv', sep=';')

In [6]:
annotation_df.head()

,VideoID,FrameNo,Phase
0,269,68,1
1,269,1043,2
2,269,1228,3
3,269,2118,4
4,269,3478,5


In [7]:
# locate high motion window in a video

# input: one video clip
# output: a score used to measure motion level (mean value of SSIM between every two adjacent frames)
def get_motion_score(video_clip, size):
    ssim_scores = np.array([])

    for _ in range(size):
        ret, frame = video_clip.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Calculate SSIM with previous frame if available
        if 'prev_frame' in locals():
            s_score = ssim(blurred, prev_frame)
        else:
            s_score = 0

        prev_frame = blurred.copy()

        ssim_scores = np.append(ssim_scores,s_score)

    return ssim_scores.mean()





In [8]:
# input: video, frame index, directory path
# output: frame tuples stored in directory
def extract_tuple(video, pos, win_size, name, 
                  dir='/Users/zhangzongkun/Desktop/research/video ordering/code/dataset/'):
    video_end = video.get(7)


    video.set(cv2.CAP_PROP_POS_FRAMES, pos)
    ret, img_a = video.read()
    if not ret:
        print('fail to read frame')
        return

    end = min(video_end, pos + win_size)
    mid = pos + (end - pos)//2

    video.set(cv2.CAP_PROP_POS_FRAMES, mid)
    ret, img_b = video.read()
    if not ret:
        print('fail to read frame')
        return
    
    video.set(cv2.CAP_PROP_POS_FRAMES, end)
    ret, img_c = video.read()
    if not ret:
        print('fail to read frame')
        return
    
    cv2.imwrite(dir+name+'_a.png', img_a)
    cv2.imwrite(dir+name+'_b.png', img_b)
    cv2.imwrite(dir+name+'_c.png', img_c)
    return


    

In [9]:
# input: video and annotation dataframe 
# output: tuples from high motion window
def pick_tuples(video, df, vid_idx, win_size = 50):
    phase_idx = 1
    phase_start, phase_end  =  df['FrameNo'][phase_idx -1], df['FrameNo'][phase_idx]

    while phase_start < phase_end:
        pos = phase_start
        motion_rcd = []
        while pos < phase_end:
            video.set(cv2.CAP_PROP_POS_FRAMES, pos)
            motion_score = get_motion_score(video, size=win_size)
            motion_rcd.append((motion_score,pos))
            pos += 50

        motion_rcd = sorted(motion_rcd, key=lambda x: x[0])
        tuple_name_0 = f'{vid_idx}_{phase_idx}_0'
        tuple_name_1 = f'{vid_idx}_{phase_idx}_1'
        extract_tuple(video, motion_rcd[0][1], win_size, tuple_name_0)
        extract_tuple(video, motion_rcd[1][1], win_size, tuple_name_1)

        phase_start = phase_end
        phase_idx += 1
        if phase_idx < len(df['FrameNo']):
            phase_end = df['FrameNo'][phase_idx]
    return

    



In [31]:
vid_idx_list = np.unique(annotation_df['VideoID'].values)


In [43]:
for vid in vid_idx_list:
    cap = cv2.VideoCapture(data_path+f'case_{vid}.mp4')
    v_df = annotation_df[annotation_df['VideoID'] == vid]
    v_df.reset_index(inplace=True)
    pick_tuples(cap, v_df, vid)

[ WARN:0@22025.629] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@22025.629] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'


[ WARN:0@22628.829] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStre

In [12]:
cap = cv2.VideoCapture(data_path+f'case_269.mp4')
v_df = annotation_df[annotation_df['VideoID'] == 269]
start_pos = v_df['FrameNo'][0]
end_pos = cap.get(7)
pos = start_pos

while pos < end_pos:
            cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
            idx = str(pos//50)
            extract_tuple(cap, pos, 50, idx,dir='/Users/zhangzongkun/Desktop/research/video ordering/code/tuple_class_demo/ordered_tuple_dataset/')
            pos += 50

[ WARN:0@83.031] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@83.031] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_506zufg7xt/croots/recipe/opencv-suite_1664548331847/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'


